### Structure of the assets folder

In [25]:
import zipfile
import boto3
import io

# See: http://boto3.readthedocs.org/en/latest/guide/configuration.html

session = boto3.Session(
    aws_access_key_id="", 
    aws_secret_access_key=""
)

s3 = session.resource("s3")
bucket = s3.Bucket('10ac-batch-6')
obj = bucket.Object('data/w11/Challenge_Data.zip')

with io.BytesIO(obj.get()["Body"].read()) as tf:

    # rewind the file
    tf.seek(0)

    # Read the file as a zipfile and process the members
    with zipfile.ZipFile(tf, mode='r') as zipf:
        for subfile in zipf.namelist():
            print(subfile)
    #$n

    

Challenge_Data/
Challenge_Data/Assets/
Challenge_Data/Assets/002dbbd85ef3fe6a2e7d0754fb9f9a1a/
Challenge_Data/Assets/002dbbd85ef3fe6a2e7d0754fb9f9a1a/0bg.png
Challenge_Data/Assets/002dbbd85ef3fe6a2e7d0754fb9f9a1a/3spraylogo.png
Challenge_Data/Assets/002dbbd85ef3fe6a2e7d0754fb9f9a1a/4bg.png
Challenge_Data/Assets/002dbbd85ef3fe6a2e7d0754fb9f9a1a/4bg1.png
Challenge_Data/Assets/002dbbd85ef3fe6a2e7d0754fb9f9a1a/5bg.png
Challenge_Data/Assets/002dbbd85ef3fe6a2e7d0754fb9f9a1a/5bg1.png
Challenge_Data/Assets/002dbbd85ef3fe6a2e7d0754fb9f9a1a/CV.jpg
Challenge_Data/Assets/002dbbd85ef3fe6a2e7d0754fb9f9a1a/_preview.png
Challenge_Data/Assets/002dbbd85ef3fe6a2e7d0754fb9f9a1a/click.png
Challenge_Data/Assets/002dbbd85ef3fe6a2e7d0754fb9f9a1a/court.png
Challenge_Data/Assets/002dbbd85ef3fe6a2e7d0754fb9f9a1a/cta.png
Challenge_Data/Assets/002dbbd85ef3fe6a2e7d0754fb9f9a1a/endframe_1.png
Challenge_Data/Assets/002dbbd85ef3fe6a2e7d0754fb9f9a1a/engagement_instruction_1.png
Challenge_Data/Assets/002dbbd85ef3fe6a2e7

In [3]:
## After downloading, unzip it and put it in the data folder
import zipfile
with zipfile.ZipFile("../Challenge_Data.zip", 'r') as zip_ref:
    zip_ref.extractall("../data/")

In [1]:
## run through the folders and subfolders, append the pathes to the images list 
## and write out the number of images traversed
import sys,os
import random

images_path = '../data/Challenge_Data/Assets/'
image_extensions = ['.jpg', '.png', '.jpeg']   # case-insensitive (upper/lower doesn't matter)
max_num_images = 10000000000

images = [os.path.join(dp, f) for dp, dn, filenames in os.walk(images_path) for f in filenames if os.path.splitext(f)[1].lower() in image_extensions]
if max_num_images < len(images):
    images = [images[i] for i in sorted(random.sample(range(len(images)), max_num_images))]

print("keeping %d images to analyze" % len(images))

keeping 16273 images to analyze


### analyzing the large set of images using a pre-trained convolutional network

In [9]:
# We will load a previously-trained neural network, that of VGG16, which comes with Keras
import keras
from keras.preprocessing import image
from keras.applications.imagenet_utils import decode_predictions, preprocess_input
from keras.models import Model
import warnings
warnings.filterwarnings('ignore')

model = keras.applications.VGG16(weights='imagenet', include_top=True)
#$n

In [22]:
## forwarding the images through the network

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

def load_image(path):
    img = tf.keras.preprocessing.image.load_img(path, target_size=model.input_shape[1:3])
    x = tf.keras.preprocessing.image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return img, x

In [11]:
feat_extractor = Model(inputs=model.input, outputs=model.get_layer("fc2").output)
feat_extractor.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [23]:
import time

tic = time.process_time()


features = []
for i, image_path in enumerate(images):
    if i % 500 == 0:
        toc = time.process_time()
        elap = toc-tic;
        print("analyzing image %d / %d. Time: %4.4f seconds." % (i, len(images),elap))
        tic = time.process_time()
    img, x = load_image(image_path);
    feat = feat_extractor.predict(x)[0]
    features.append(feat)

print('finished extracting features for %d images' % len(images))

analyzing image 0 / 16273. Time: 0.0001 seconds.


AttributeError: module 'keras.preprocessing.image' has no attribute 'img_to_array'